<div class="demo-info">

# Agile Metabolic Health Blood Pressure Dashboard Demo

Note: this cell has custom styling that makes it **not** show up in the Voilà Dashboard. 
You can hide any other markdown in this notebook by surrounding it with the same "demo-info"
class ID as this one.

This dashboard fetches and displays blood pressure data for a patient.

(DEMO) To edit the dashboard:

1. <a href="#" id="demo-edit-link" target="demo-edit">click here</a> to open the notebook
1. edit the notebook
1. save changes
1. reload this page

To add goals to the demo, add `include_goal=True` to the last cell in this notebook to select a patient, edit the cell below that sets `patient_id` , `study_id`. You can see values in the [CHCS Portal](https://chcs.fly.dev/portal/studies).

</div>

<script type="text/javascript">
var link = document.getElementById("demo-edit-link");
link.href = document.location.href.replace("/voila/render/", "/lab/tree/")
</script>

<style type="text/css">
.demo-info {
    font-style: italic;
    /* hide all the demo info */
    display: none;
}

.added-widget {
    /* add some highlighting for widgets added in the demo */
    font-weight: bold;
    background-color: rgb(220, 255, 220) !important;
}

</style>

In [1]:
import os
from IPython.display import display, HTML, Image, Markdown

# Convenience function to display markdown
md = lambda s: display(Markdown(s))

# Use "if not VOILA:" blocks to print debug/private output
# Otherwise all output is by default shown by VOILA
VOILA = os.environ.get("VOILA_REQUEST_URL")

In [2]:
from jupyter_health import Code, JupyterHealthCHClient

client = JupyterHealthCHClient()
BLOOD_PRESSURE = Code.BLOOD_PRESSURE.value

In [3]:
# check our credentials:
try:
    user_info = client.get_user()
except Exception as e:
    msg = "Failed to access CHCS endpoint. Logout of JupyterHub, login again, and restart your server to get fresh credentials."
    print(msg)
    print(e)
    raise PermissionError(msg)

if not VOILA:
    print("logged in as:")
    display(user_info)

logged in as:


{'id': 10054,
 'email': 'fernando.perez@berkeley.edu',
 'firstName': 'Fernando',
 'lastName': 'Perez',
 'patient': None}

In [4]:
# select paiient_id
# use user's own patient id if they are a patient,
# otherwise a test id that has data
user_patient = user_info["patient"]
if user_patient:
    # user is a patient, show them their own data
    patient_id = int(user_patient)
    study_id = None
else:
    study_id   = 30007  # Jupyter Study
    patient_id = 40072  # Fernando's ID

In [5]:
patients = list(client.list_patients())

if not VOILA:
    print("available patients:")
    using_patient = None
    for patient in patients:
        if patient['id'] == patient_id:
            using_patient = patient
        # print("id:", resource['id'])
        # print("Given name:", *resource['name'][0]['given'])
        # print("Family name:", resource['name'][0]['family'])
        email = patient.get("telecomEmail", None)
        if email:
            print(patient["id"], email, f"{patient['nameFamily']}, {patient['nameGiven']}")
    print()
    print(f"Using patient: {using_patient['id']}; {using_patient['nameFamily']}, {using_patient['nameGiven']}")

available patients:
40001 peter@example.com Patterson, Peter
40002 nika@example.com Lin, Nika
40004 lee@example.com Rulfs, Lee
40049 2e33@emiail.com ds, ds
40052 tues@example.com Smith, Tuesday
40056 benben03129@email.com ben, ben1
40057 bennbenb01923@email.com benben, benebenebn
40058 fperez_patient@example.com Perez, Fernando
40060 uniqueemial1239123@email.com testname, testname
40061 ucsftestpatient1@email.com test, test
40062 bentest111111@email.com bentest1, bentest1
40063 fernandoemail@email.com fernando1, fernando2
40064 basdfjals@email.com ben1, ben1
40065 benemailtest23@email.com bentest, bentest
40066 bennyemail123@email.com ben123, ben123
40069 fernando@email.com Fernando1, Fernando1
40070 bentest91232@email.com bentest, bentest1
40071 benben123@email.com benben, benben
40072 fernando2test@email.com fernando2, fernando2

Using patient: 40072; fernando2, fernando2


In [6]:
df = patient_df = client.list_observations_df(patient_id=patient_id, 
                                              study_id=study_id, code=Code.BLOOD_PRESSURE)

if not len(patient_df):
    raise ValueError("Found no observations!")

if not VOILA:
    display(patient_df.head())

,resource_type,resourceType,id,meta_lastUpdated,identifier_0_value,identifier_0_system,status,subject_reference,code_coding_0_code,code_coding_0_system,...,source_data_point_id,source_creation_date_time,effective_time_frame_date_time,systolic_blood_pressure_unit,systolic_blood_pressure_value,diastolic_blood_pressure_unit,diastolic_blood_pressure_value,creation_date_time_local,source_creation_date_time_local,effective_time_frame_date_time_local
0,omh:blood-pressure:4.0,Observation,64126,2024-10-30T17:48:52.463857+00:00,0d202c6b-13ba-33e2-a140-5b0920da91af,https://chcs.org,final,Patient/40072,omh:blood-pressure:4.0,https://w3id.org/openmhealth,...,1e8f7467def3dc812719f1674ec13ca3,2024-10-30 13:51:44+00:00,2024-10-30 13:51:44+00:00,mmHg,145,mmHg,100,2024-10-30 17:48:46.301,2024-10-30 06:51:44,2024-10-30 06:51:44
1,omh:blood-pressure:4.0,Observation,64125,2024-10-29T18:59:44.209601+00:00,3ca003d4-7d68-3e31-8deb-a2124c9c71d2,https://chcs.org,final,Patient/40072,omh:blood-pressure:4.0,https://w3id.org/openmhealth,...,e3911798f34393af6ad02f3c0e3025ee,2024-10-29 05:04:05+00:00,2024-10-29 05:04:05+00:00,mmHg,150,mmHg,94,2024-10-29 18:59:25.962,2024-10-28 22:04:05,2024-10-28 22:04:05
2,omh:blood-pressure:4.0,Observation,64124,2024-10-29T18:59:44.195023+00:00,2809a3b9-eb6a-35fe-b768-54edbea55201,https://chcs.org,final,Patient/40072,omh:blood-pressure:4.0,https://w3id.org/openmhealth,...,654450a2182dae5f6c5849b1b884f307,2024-10-26 17:32:19+00:00,2024-10-26 17:32:19+00:00,mmHg,127,mmHg,90,2024-10-29 18:59:25.962,2024-10-26 10:32:19,2024-10-26 10:32:19
3,omh:blood-pressure:4.0,Observation,64123,2024-10-29T18:59:44.1815+00:00,e673bf51-f9ce-351a-8266-1f15d257450c,https://chcs.org,final,Patient/40072,omh:blood-pressure:4.0,https://w3id.org/openmhealth,...,78ed7d5057a57d9c2034e52e7fb35069,2024-10-25 14:05:03+00:00,2024-10-25 14:05:03+00:00,mmHg,140,mmHg,97,2024-10-29 18:59:25.962,2024-10-25 07:05:03,2024-10-25 07:05:03
4,omh:blood-pressure:4.0,Observation,64122,2024-10-29T18:59:44.169269+00:00,ec25bd78-86d5-3e2a-8dd6-eeb7cc22dc23,https://chcs.org,final,Patient/40072,omh:blood-pressure:4.0,https://w3id.org/openmhealth,...,e03ddf8ca696a93f98135fc8b5b8d391,2024-10-25 04:07:33+00:00,2024-10-25 04:07:33+00:00,mmHg,143,mmHg,97,2024-10-29 18:59:25.962,2024-10-24 21:07:33,2024-10-24 21:07:33


In [7]:
# now start the plotting

from datetime import timedelta
from enum import Enum
from functools import partial
from itertools import chain

import altair as alt
import pandas as pd

pd.options.mode.chained_assignment = None


class Goal(Enum):
    """Enum for met/unmet

    These strings will be used for the legend.
    """

    met = "under"
    unmet = "over"


class Category(Enum):
    normal = "normal"
    elevated = "elevated"
    hypertension = "hypertension"


def classify_bp(row):
    """Classify blood pressure"""
    # https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings
    # note from : We can decide to have just normal, elevated and hypertension to begin with
    if (
        row.diastolic_blood_pressure_value < 80
        and row.systolic_blood_pressure_value < 120
    ):
        return Category.normal.value
    elif (
        row.diastolic_blood_pressure_value < 80
        and 120 <= row.systolic_blood_pressure_value < 130
    ):
        return Category.elevated.value
    else:
        return Category.hypertension.value


def bp_goal(patient_df, goal="140/90"):
    """True/False for blood pressure met goal"""
    sys_goal, dia_goal = (int(s) for s in goal.split("/"))
    if (patient_df.systolic_blood_pressure_value <= sys_goal) & (
        patient_df.diastolic_blood_pressure_value <= dia_goal
    ):
        return Goal.met.value
    else:
        return Goal.unmet.value


red_yellow_blue = [
    "#4a74b4",
    "#faf8c1",
    "#d4322c",
]
red_blue = [red_yellow_blue[0], red_yellow_blue[-1]]


def bp_over_time(bp, color_scale="category"):
    """Plot blood pressure over time"""
    # https://vega.github.io/vega/docs/schemes/#redyellowblue
    if color_scale == "category":
        domain = [
            Category.normal.value,
            Category.elevated.value,
            Category.hypertension.value,
        ]
        color = alt.Color(
            "category:O",
            scale=alt.Scale(
                domain=domain,
                range=red_yellow_blue,
            ),
        )
        shape = alt.Shape(
            "category:O",
            scale=alt.Scale(
                domain=domain,
            ),
        )
    elif color_scale == "goal":
        domain = [Goal.met.value, Goal.unmet.value]
        color = alt.Color(
            "goal:O",
            scale=alt.Scale(domain=domain, range=red_blue),
        )
        shape = alt.Shape(
            "goal:O",
            scale=alt.Scale(domain=domain),
        )

    # heuristic for x-ticks
    end_time = bp.effective_time_frame_date_time.max()
    end_date = end_time.date()
    start_date = bp.effective_time_frame_date_time.min().date()
    time_frame_days = (end_date - start_date).total_seconds() / (3600 * 24)
    axis_args = {"format": "%Y-%m-%d"}
    if time_frame_days < 7:
        # minimum of one week
        start_date = end_date - timedelta(days=7)
    if time_frame_days < 14:
        # at least a week
        axis_args["tickCount"] = dict(interval="day", step=1)
    if 14 <= time_frame_days < 30:
        # expand less than a month to 1 month
        start_date = end_date - timedelta(days=30)
    if time_frame_days > 90:
        # at least a few months, label with year-month
        axis_args["format"] = "%Y-%m"

    x = alt.X(
        "effective_time_frame_date_time_local",
        title="date",
        axis=alt.Axis(
            labelAngle=30,
            **axis_args,
        ),
        scale=alt.Scale(
            domain=[
                pd.to_datetime(start_date),
                pd.to_datetime(end_date + timedelta(days=1)),
            ]
        ),
    )

    charts = [
        [
            alt.Chart(bp, title="blood pressure")
            .mark_line(color="#333")
            .encode(
                x=x,
                y=alt.Y(f"{which}_blood_pressure_value", title="mmHg"),
            ),
            alt.Chart(bp, title="blood pressure")
            .mark_point(filled=True)
            .encode(
                x=x,
                y=alt.Y(f"{which}_blood_pressure_value", title="mmHg"),
                color=color,
                shape=shape,
                tooltip=[
                    alt.Tooltip("effective_time_frame_date_time_local", title="date"),
                    alt.Tooltip("systolic_blood_pressure_value", title="Systolic"),
                    alt.Tooltip("diastolic_blood_pressure_value", title="Diastolic"),
                    alt.Tooltip("category"),
                ],
            ),
        ]
        for which in ("systolic", "diastolic")
    ]
    return alt.layer(*chain(*charts))


def bp_by_tod(bp):
    """Plot blood pressure by time of day"""
    tod_tooltip = [
        alt.Tooltip(
            "mean(diastolic_blood_pressure_value):Q",
            title="avg diastolic",
            format=".0f",
        ),
        alt.Tooltip(
            "mean(systolic_blood_pressure_value):Q", title="avg systolic", format=".0f"
        ),
        alt.Tooltip("count(diastolic_blood_pressure_value)", title="measurements"),
        alt.Tooltip("hours(effective_time_frame_date_time_local)", title="hour"),
    ]
    charts = [
        alt.Chart(bp, title="by time of day")
        .mark_line(point=True)
        .encode(
            x=alt.X(
                "hours(effective_time_frame_date_time_local)",
                title="time of day",
                scale=alt.Scale(domain=[0, 24]),
            ),
            y=alt.Y(
                f"mean({which}_blood_pressure_value):Q",
                title=which,
                axis=alt.Axis(title=""),
            ),
            tooltip=tod_tooltip,
        )
        for which in ("systolic", "diastolic")
    ]
    return alt.layer(*charts)


def plot_patient_blood_pressure(patient_df, goal="140/90", color_scale="category"):
    """plot blood pressure, given a patient data frame, as returned by get_patient_data"""
    bp = patient_df.loc[patient_df.resource_type == BLOOD_PRESSURE]
    bp["category"] = bp.apply(classify_bp, axis=1)
    bp["goal"] = bp.apply(partial(bp_goal, goal=goal), axis=1)
    return (
        (bp_over_time(bp, color_scale=color_scale) | bp_by_tod(bp))
        .resolve_scale(y="shared", color="independent", shape="independent")
        .configure_point(size=100)
        .interactive()
    )


if not VOILA:
    # show last 30 days of data
    end_date = df.effective_time_frame_date_time.max()
    display(
        plot_patient_blood_pressure(
            df[df.effective_time_frame_date_time >= (end_date - timedelta(days=30))],
            color_scale="goal",
        )
    )

alt.HConcatChart(...)

In [8]:
import ipywidgets as W


def bp_category_style(row):
    """highlight rows by bp category"""
    category = Category(row.category)
    if category == Category.hypertension:
        color = "#fdd"
    elif category == Category.elevated:
        color = "#ffc"
    else:
        color = None

    return [f"background-color:{color}" if color else None] * len(row)


def bp_goal_style(row):
    """highlight rows by bp category"""
    goal = Goal(row.goal)
    if goal == Goal.unmet:
        color = "#fdd"
    else:
        color = None

    return [f"background-color:{color}" if color else None] * len(row)


def bp_goal_fraction(patient_df, goal="140/90"):
    bp = patient_df.loc[patient_df.resource_type == BLOOD_PRESSURE]
    sys_goal, dia_goal = (int(s) for s in goal.split("/"))
    met_goal = (bp.systolic_blood_pressure_value <= sys_goal) & (
        bp.diastolic_blood_pressure_value <= dia_goal
    )
    return met_goal.sum() / len(bp)


def bp_table_info(patient_df, goal="140/90", color_scale="category"):
    """Display tabular info"""
    bp = patient_df.loc[
        patient_df.resource_type == BLOOD_PRESSURE,
        [
            "effective_time_frame_date_time_local",
            "systolic_blood_pressure_value",
            "diastolic_blood_pressure_value",
        ],
    ]
    bp["category"] = bp.apply(classify_bp, axis=1)
    bp["goal"] = bp["goal"] = bp.apply(partial(bp_goal, goal=goal), axis=1)
    # relabel columns
    bp.columns = ["date", "systolic", "diastolic", "category", "goal"]
    bp["time"] = bp.date.dt.time
    bp["date"] = bp.date.dt.date
    bp = bp.astype({"systolic": int, "diastolic": int})

    label_style = {"font_weight": "bold", "font_size": "150%"}

    table = W.Output()
    with table:
        styled = (
            bp.style.hide()
            .hide(["category", "goal"], axis="columns")
            .format({"time": "{:%H:%M}"})
        )
        if color_scale == "goal":
            styled = styled.apply(bp_goal_style, axis=1)
        else:
            styled = styled.apply(bp_category_style, axis=1)
        display(HTML(styled.to_html(index=False)))

    summary = W.Output()
    min_idx = bp.systolic.idxmin()
    max_idx = bp.systolic.idxmax()
    summary_table = pd.DataFrame(
        {
            "systolic": [
                bp.systolic.min(),
                bp.systolic.max(),
                bp.systolic.mean(),
            ],
            "diastolic": [
                bp.diastolic.min(),
                bp.diastolic.max(),
                bp.diastolic.mean(),
            ],
            "date": [
                bp.loc[min_idx].date,
                bp.loc[max_idx].date,
                "-",
            ],
        }
    )
    summary_table = summary_table.astype({"systolic": int, "diastolic": int})
    summary_table.index = pd.Index(["min", "max", "avg"])
    with summary:
        display(HTML(summary_table.to_html()))

    # calculate goal fraction
    at_goal_fraction = bp_goal_fraction(patient_df, goal)
    overview = W.Output()
    with overview:
        display(
            HTML(f"<div style='font-size: 250%'>at goal: {at_goal_fraction:.0%}</div>")
        )

    box_layout = {
        "border_left": "1px solid #aaa",
        "padding": "8px",
        "margin": "8px",
    }
    right_box = [
        W.Label(value="summary", style=label_style),
        summary,
    ]
    if color_scale == "goal":
        right_box.extend(
            [
                W.Label(value="overview", style=label_style),
                overview,
            ]
        )
    layout = W.HBox(
        [
            W.VBox(
                [W.Label(value="Measurements", style=label_style), table],
                layout=box_layout,
            ),
            W.VBox(
                right_box,
                layout=box_layout,
            ),
        ]
    )
    layout.layout.justify_content = "flex-start"
    return layout


if not VOILA:
    # preview while run interactively (not voila)
    display(bp_table_info(df[-100:], color_scale="goal"))

In [9]:
import time


def plot_patient(
    view=30,
    comorbidity=False,
    color="goal",
):
    if comorbidity:
        goal = "130/80"
    else:
        goal = "140/90"
    df = patient_df

    last_day = df.effective_time_frame_date_time.dt.date.max()
    start_date = last_day - timedelta(days=view)
    df = df[df.effective_time_frame_date_time.dt.date >= start_date]
    # for demo: scale if current user is too healthy
    df = df.copy()
    df["systolic_blood_pressure_value"] = (
        df["systolic_blood_pressure_value"]
    ).astype(int)
    df["diastolic_blood_pressure_value"] = (
        df["diastolic_blood_pressure_value"]
    ).astype(int)
    display(plot_patient_blood_pressure(df, goal=goal, color_scale=color))
    display(bp_table_info(df, goal=goal, color_scale=color))
    # utterly bizarre: voilá hangs if this returns too quickly
    time.sleep(0.5)


if not VOILA:
    plot_patient()

alt.HConcatChart(...)

<div class="demo-info">

---
    
Everything above here is setup of the demo itself, not part of the demo; this would all be hidden and managed by EHR inputs.

Below here is the actual demo that is meant to show up as a dashboard and meant for display.

# Dashboard Creation

</div>

In [10]:
# create the interactive chart

def create_dashboard(include_goal=False):
    """
    Create the dashboard

    only one input: include_goal
    if True, include UCSF goal inputs ('after' view)
    if False, only include American Heart Association category view
    """

    interact_args = {}
    if include_goal:
        comorbidity_widget = W.Checkbox(
            description="Any comorbidities: ASCVD > 10%, Diabetes Mellitus, CKD (EGFR 20-59), Heart Failure"
        )
        comorbidity_widget.layout.width = "100%"
        comorbidity_widget.add_class("added-widget")
        color_widget = W.Dropdown(
            value="goal", options={"UCSF Goal": "goal", "AHA Category": "category"}
        )
        color_widget.add_class("added-widget")
        interact_args["comorbidity"] = comorbidity_widget
        interact_args["color"] = color_widget
    else:
        interact_args["comorbidity"] = W.fixed(False)
        interact_args["color"] = W.fixed("category")

    dashboard = W.interactive(
        plot_patient,
        view={
            "Week": 7,
            "Month": 30,
            "Year": 365,
        },
        **interact_args,
    )
    # give it some border to set it off from the demo setup
    dashboard.layout.border = "4px solid #cff"
    dashboard.layout.padding = "16px 30px"

    # rerender on change of widgets outside the interact
    # patient_widget.observe(lambda change: dashboard.update(), names="value")
    # partner_widget.observe(lambda change: dashboard.update(), names="value")
    return dashboard

In [11]:
Image(url="https://jupyterhealth.org/images/PoweredByJupyter.png")


# Agile Metabolic JupyterHealth Dashboard


welcome NIH!

In [12]:
md(f"#### Logged in as administrative user:")
md(f"{user_info['firstName']} {user_info['lastName']} ({user_info['email']}).")
md("---")

#### Logged in as administrative user:

Fernando Perez (fernando.perez@berkeley.edu).

---

In [13]:
# DEMO: uncomment include_goal=True to add UCSF goals to visualization,
create_dashboard(
    include_goal=True,
)

interactive(children=(Dropdown(description='view', index=1, options={'Week': 7, 'Month': 30, 'Year': 365}, val…